
<p> <center> <a href="../Start_Here.ipynb">Home Page</a> </center> </p>

 
<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="3.Hands-on-Multi-GPU.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 33%; text-align: center;">
        <a href="1.Introduction-to-Distributed-Deep-Learning.ipynb">1</a>
        <a href="2.1.System-Topology.ipynb">2</a>
        <a href="3.Hands-on-Multi-GPU.ipynb">3</a>
        <a >4</a>
    </span>
</div>



# Introduction to Distributed Deep Learning - Part 4

**Contents of this notebook:**

- [**Challenges with convergence**](#Challenges-with-convergence)
    - [Concepts](#Concepts)
        - [Impact of Batch size](#Impact-of-Batch-size)
        - [Impact on test and validation accuracy](#Impact-on-test-and-validation-accuracy)
    - [Techniques for faster convergence](#Techniques-for-faster-convergence)
        - [Batch norm](#Batch-norm)
        - [Learning rate scaling](#Learning-rate-scaling)
        - [Learning rate warmup](#Learning-rate-warmup)
        - [Using Optimizers built for Exascale Deep learning](#Using-Optimizers-built-for-Exascale-Deep-learning)

**By the End of this Notebook you will:**

- Understand the reasons behind slower convergence with increase in Batch size. 
- Learn techniques to make convergence faster.

# Challenges with convergence

We noticed in the previous notebook that the Distributed Training with 2 or more GPUs had higher throughput than the single GPU training, but when we focused on the convergence of our model, we found it to converge slower than the former.
*Quoted from the previous notebook* : If we take a closer look at the results, we can find even after 8 epochs in both cases, the run with a single GPU at the end of 8 epochs has a **loss of 0.0487 and accuracy of 0.9851** , comparing that with 8 GPU case, we find at the end of 8 epochs we have a **loss of 0.3582 and accuracy of 0.8957**.

We were also able to notice the accuracy took a bigger drop during training, let us now try to understand why this happens and how we can use some techniques to solve them.

## Concepts


### Impact of Batch size
In the paper **Measuring the Effects of Data Parallelism on Neural Network Training** by **Christopher J. Shallue, Jaehoon Lee, Joseph Antognini, Jascha Sohl-Dickstein, Roy Frostig, George E. Dahl** , we see a relationship between steps taken to convergence and the batch size. 


<center><img src="images/paper1.png" width="700"/></center>


To understand this phenomenon,a term called as Critical batch size was coined in the paper **An Empirical Model of Large-Batch Training** by **Sam McCandlish, Jared Kaplan, Dario Amodei, OpenAI Dota Team**

#### Critical batch size 

Critical batch size is when compute efficiency drops below 50% optimal and larger batch sizes yield diminishing returns. 


It is found that we can approximately predict the maximum useful batch size by measuring gradient noise scaling, which is a simple statistic that quantifies the signal-to-noise ratio of the network gradients. Heuristically, the noise scale measures the variation in the data as seen by the model (at a given stage in training). When the noise scale is small, looking at a lot of data in parallel quickly becomes redundant, whereas when it is large, we can still learn a lot from huge batches of data.

Below is the critical batch size of some popular networks.

<center><img src="images/noise-summary-3.svg" width="720"/></center>


Let us now understand another problem that we face in large-batch training. 

### Impact on test and validation accuracy 

It is found that test/validation accuracy decreases with an increase in Batch size, this can be noticed in our previous notebook as well from the figure below, which is from the paper **ImageNet Training in Minutes** by **Yang You, Zhao Zhang, Cho-Jui Hsieh, James Demmel, Kurt Keutzer** 

<center><img src="images/accuracy.png"/></center>

This lack of generalization ability is because large-batch methods tend to converge to sharp minimizers of the training function. These minimizers are characterized by a significant number of large positive eigenvalues in $\nabla^2 f(x)$ , and tend to generalize less well. In contrast, small-batch methods converge to flat minimizers characterized by having numerous small eigenvalues of $\nabla^2 f(x)$. We have observed that the loss function landscape of deep neural networks is such that large-batch methods are attracted to regions with sharp minimizers. Unlike small-batch methods, are unable to escape basins of attraction of these minimizers.

<center><img src="images/minima.png" width="600"/></center>

This convergences to sharp minima and the difference between the training and test function lead to a higher validation loss. 

Now that we've understood the challenges with regards to convergence, let us now see some techniques to improve the convergence rate.

**Before that let us now use a different dataset , We will be using CIFAR-10, which is a slightly more complex compared to FMNIST as it has 3 channel per image ( RGB ) while FMNIST only has 1.**

Let us start with benchmarking CIFAR-10 with single and 8 GPUs and compare the convergence using train accuracy.

In [ ]:
!TF_CPP_MIN_LOG_LEVEL=3 horovodrun -np 1 python3 ../source_code/N4/cifar_base.py --batch-size=512 2> /dev/null

In [ ]:
!TF_CPP_MIN_LOG_LEVEL=3 horovodrun -np 8 --mpi-args="--oversubscribe" python3 ../source_code/N4/cifar_base.py --batch-size=512 2> /dev/null

#### Results

Output of running the command on DGX-1 : 


Single GPU : 

```bash
[1,0]<stdout>:Epoch 11/12
[1,0]<stdout>:97/97 [==============================] - 1s 14ms/step - loss: 0.8486 - accuracy: 0.7002
[1,0]<stdout>:Epoch time : 1.3384358882904053
[1,0]<stdout>:Images/sec: 37357.04
[1,0]<stdout>:Epoch 12/12
[1,0]<stdout>:97/97 [==============================] - 1s 14ms/step - loss: 0.8085 - accuracy: 0.7153
[1,0]<stdout>:Epoch time : 1.3324604034423828
[1,0]<stdout>:Images/sec: 37524.57
```

Scaling with 8-GPUs : 

```bash
[1,0]<stdout>:Epoch 11/12
[1,0]<stdout>:12/12 [==============================] - 0s 19ms/step - loss: 1.5277 - accuracy: 0.4408
[1,0]<stdout>:Epoch time : 0.23624348640441895
[1,0]<stdout>:Images/sec: 211646.05
[1,0]<stdout>:Epoch 12/12
[1,0]<stdout>:12/12 [==============================] - 0s 18ms/step - loss: 1.5352 - accuracy: 0.4404
[1,0]<stdout>:Epoch time : 0.22572755813598633
[1,0]<stdout>:Images/sec: 221505.96
```

Now that we have a baseline , let us now try to improve the convergence rate of the models.

# Techniques for faster convergence

### Batch normalisation

In the paper **Train longer, generalize better: closing the generalization gap in large batch training of neural networks** by **Elad Hoffer, Itay Hubara, Daniel Soudry** they introduced Ghost Batch Normalization to reduce generalization error.

Batch Normalization is known to accelerate the training, increase the robustness of the neural network to different initialization schemes and improve generalization. Nonetheless, since it uses batch statistics, it is bounded to depend on the chosen batch size. We study this dependency and observe that by acquiring the statistics on small virtual ("ghost") batches instead of the real large batch, we can reduce the generalization error. This modification by itself reduces the generalization error substantially.

This can be implemented by the Normalization of a smaller batch present in each GPU instead of the larger batch across all GPUs.

Let us now implement Batch Norm for every GPU and test out that performance.

This can be implemented by adding `BatchNormalisation` layers to the model using the Keras API 

```python3
X = BatchNormalization(axis=3)(X)
```

In [ ]:
!TF_CPP_MIN_LOG_LEVEL=3 horovodrun -np 8 --mpi-args="--oversubscribe" python3 ../source_code/N4/cifar_batch_norm.py --batch-size=512 2> /dev/null

#### Result

Output of running the command on DGX-1 : 

```bash
[1,0]<stdout>:Epoch 11/12
[1,0]<stdout>:12/12 [==============================] - 0s 22ms/step - loss: 1.0192 - accuracy: 0.6413
[1,0]<stdout>:Epoch time : 0.2735874652862549
[1,0]<stdout>:Images/sec: 182756.91
[1,0]<stdout>:Epoch 12/12
[1,0]<stdout>:12/12 [==============================] - 0s 24ms/step - loss: 1.0059 - accuracy: 0.6416
[1,0]<stdout>:Epoch time : 0.2887394428253174
[1,0]<stdout>:Images/sec: 173166.5
```

### Learning rate scaling

In the research paper **Accurate, Large Minibatch SGD:Training ImageNet in 1 Hour** by **Facebook researchers** they introduced the Linear Scaling Rule, which states that  **When the minibatch size is multiplied by k, multiply the learning rate by k.**

This can be implemented by Horovod using the following line. 

```python 
scaled_lr = 0.001 * hvd.size()       # Scale Learning rate to number of GPUs 
opt = tf.optimizers.Adam(scaler_lr)
opt = hvd.DistributedOptimizer(opt)  # Wrap Optimizer with hvd.DistributedOptimizer()
```

Now, let us try running the cell with a scaled learning rate. 

In [ ]:
!TF_CPP_MIN_LOG_LEVEL=3 horovodrun -np 8 --mpi-args="--oversubscribe" python3 ../source_code/N4/cifar_scalelr.py --batch-size=512 2> /dev/null

### Result 

Output of running the command on DGX-1 with 8 GPUs : 


```bash
[1,0]<stdout>:Epoch 11/12
[1,0]<stdout>:12/12 [==============================] - 0s 25ms/step - loss: 1.2916 - accuracy: 0.5309
[1,0]<stdout>:Epoch time : 0.3019692897796631
[1,0]<stdout>:Images/sec: 165579.75
[1,0]<stdout>:Epoch 12/12
[1,0]<stdout>:12/12 [==============================] - 0s 24ms/step - loss: 1.2914 - accuracy: 0.5295
[1,0]<stdout>:Epoch time : 0.2958521842956543
[1,0]<stdout>:Images/sec: 169003.32
```

We might notice that this time training accuracy did not converge , this is because the linear scaling rule breaks down when the network is changing rapidly, which commonly occurs in early stages of training. We find that this issue can be alleviated by a properly designed warmup. 

**Note** : You might get lucky and get it might converge , so try running the above cell multiple times to notice the effect of larger batch size.

### Learning rate warmup

Learning rate warmup gradually ramps up the learning rate from a small to a large value. This ramp avoids a sudden increase from a small learning rate to a large one, allowing healthy convergence at the start of training. In practice, with a large minibatch of size kn, we start from a learning rate of η and increment it by a constant amount at each iteration such that it reaches ηˆ = kη after 5 epochs. After the warmup phase, we go back to the original learning rate schedule. 

Learning rate warmup can be implemented in Horovod using the following callback.

```python
# Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final accuracy.
# Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during the first three epochs.
hvd.callbacks.LearningRateWarmupCallback(initial_lr=scaled_lr, warmup_epochs=3, verbose=1)
```

Let us now implement this and run this.

In [ ]:
!TF_CPP_MIN_LOG_LEVEL=3 horovodrun -np 8 --mpi-args="--oversubscribe" python3 ../source_code/N4/cifar_warmup.py --batch-size=512 2> /dev/null

### Result 

Output of running the command on DGX-1 : 


```bash
[1,0]<stdout>:Epoch 11/12
[1,0]<stdout>:12/12 [==============================] - 0s 23ms/step - loss: 1.0463 - accuracy: 0.6343
[1,0]<stdout>:Epoch time : 0.27834606170654297
[1,0]<stdout>:Images/sec: 179632.5
[1,0]<stdout>:Epoch 12/12
[1,0]<stdout>:12/12 [==============================] - 0s 23ms/step - loss: 1.0108 - accuracy: 0.6450
[1,0]<stdout>:Epoch time : 0.28165721893310547
[1,0]<stdout>:Images/sec: 177520.75
```


In this specific case , we obtain the results close to Batch Normalisation , but with increase in the batch size to a much higher value ,we can notice the improvements provided by Learning rate scaling and warmup.

### Using Optimizers built for Exascale Deep learning

#### The LAMB Optimizer

A series of optimizers have been created to address this problem and allow for scaling to very large batch sizes and learning rates. One such optimizer is the LAMB optimizer. LAMB uses a layerwise adaptive large batch optimization to train with very little hyperparameter tuning. This optimizer enables the use of very large batch sizes without any degradation of performance.

We can implement LAMB using the following lines.

```python
# Import LAMB from Tensorflow Addons
from tensorflow_addons.optimizers import LAMB

# Replace the Adam optimizer with LAMB:
opt = LAMB(learning_rate=scaled_lr)
```

Let us now train using the LAMB optimizer and compare the results. 

You can learn more about the LAMB optimizer from its paper [here](https://arxiv.org/pdf/1904.00962.pdf)

In [ ]:
!TF_CPP_MIN_LOG_LEVEL=3 horovodrun -np 8 --mpi-args="--oversubscribe" python3 ../source_code/N4/cifar_lamb.py --batch-size=512 2> /dev/null

### Result 

Output of running the command on DGX-1 : 

```bash
[1,0]<stdout>:Epoch 11/12
[1,0]<stdout>:12/12 [==============================] - 0s 33ms/step - loss: 0.6223 - accuracy: 0.7829
[1,0]<stdout>:Epoch time : 0.4006154537200928
[1,0]<stdout>:Images/sec: 124807.97
[1,0]<stdout>:Epoch 12/12
[1,0]<stdout>:12/12 [==============================] - 0s 34ms/step - loss: 0.5630 - accuracy: 0.8092
[1,0]<stdout>:Epoch time : 0.4174768924713135
[1,0]<stdout>:Images/sec: 119767.11
```

Let us now tabulate all the results together to understand the improvements in convergence time.

|# of GPUs|Condition|Accuracy after 12 epochs|
|-|-|-|
|1|Single GPU|71%|
|8|Multi-GPU naive|44%|
|8|Batch Normalisation|64%|
|8|Learning rate scaling+warmup|65%|
|8|LAMB Optimizer|81%|


Now that we are aware of the concepts in Distributed deep learning training and techniques to use for faster convergence, Go to the next notebook to run the challenge notebook.

***

## Licensing

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).

<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="3.Hands-on-Multi-GPU.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 33%; text-align: center;">
        <a href="1.Introduction-to-Distributed-Deep-Learning.ipynb">1</a>
        <a href="2.1.System-Topology.ipynb">2</a>
        <a href="3.Hands-on-Multi-GPU.ipynb">3</a>
        <a >4</a>
    </span>
</div>

<br>

<p> <center> <a href="../Start_Here.ipynb">Home Page</a> </center> </p>
